# 🎯 Backtracking Mastery

**Phase 3: Interview Preparation - Coding Problems Module 4**

**Prerequisites**: Recursion, state management, understanding decision trees

**Master systematic exploration of solution spaces - essential for constraint satisfaction problems!**

---

## 🏗️ Backtracking Fundamentals

Understanding decision trees, state space, and pruning strategies

In [ ]:
// Backtracking Framework and Analysis
case class BacktrackResult[T](
  solutions: List[T],
  nodesExplored: Int,
  maxDepth: Int,
  branchesPruned: Int,
  timeComplexity: String,
  constraintChecks: Int
)

// Decision tree states
sealed trait DecisionState
case object Promising extends DecisionState
case object Unpromising extends DecisionState
case object Solution extends DecisionState

// Generic backtracking template
class BacktrackingEngine[T, S](
  initialState: S,
  goalPredicate: S => Boolean,
  constraintFilter: S => Boolean,
  branchingStrategy: S => List[S],
  solutionExtractor: S => T
) {
  private var nodesExplored = 0
  private var maxDepthReached = 0
  private var branchesPruned = 0
  private var constraintChecks = 0
  
  def findAllSolutions(): BacktrackResult[T] = {
    val solutions = collection.mutable.ListBuffer[T]()
    
    def backtrack(currentState: S, depth: Int): Unit = {
      nodesExplored += 1
      maxDepthReached = math.max(maxDepthReached, depth)
      
      // Check if solution found
      if (goalPredicate(currentState)) {
        solutions.append(solutionExtractor(currentState))
        return
      }
      
      // Generate candidate extensions
      val candidates = branchingStrategy(currentState)
      
      for (candidate <- candidates) {
        constraintChecks += 1
        
        if (constraintFilter(candidate)) {
          // Continue exploring
          backtrack(candidate, depth + 1)
        } else {
          branchesPruned += 1
        }
      }
    }
    
    backtrack(initialState, 0)
    
    BacktrackResult(
      solutions.toList,
      nodesExplored,
      maxDepthReached,
      branchesPruned,
      s"O(${nodesExplored}_nodes)",
      constraintChecks
    )
  }
  
  def getStats(): (Int, Int, Int, Int) = (
    nodesExplored, maxDepthReached, branchesPruned, constraintChecks
  )
}

// Pattern recognition for backtracking problems
object BacktrackingPatterns {
  
  def identifyPattern(problem: String): BacktrackingPattern = {
    val desc = problem.toLowerCase
    if (desc.contains("permutation") || desc.contains("combination")) CombinationPattern
    else if (desc.contains("n-queens") || desc.contains("sudoku")) ConstraintSatisfaction
    else if (desc.contains("subset") || desc.contains("subsequence")) SubsetGeneration
    else if (desc.contains("palindrome") || desc.contains("path")) StringPathPattern
    else if (desc.contains("generate") || desc.contains("all possible")) EnumerationPattern
    else UnknownPattern
  }
  
  def getTimeComplexity(pattern: BacktrackingPattern, n: Int): String = pattern match {
    case CombinationPattern => s"O(2^$n) worst case"
    case ConstraintSatisfaction => s"O(k^$n) where k is domain size"
    case SubsetGeneration => s"O(2^$n)"
    case StringPathPattern => s"O(2^$n) worst case"
    case EnumerationPattern => s"O(c^$n)"
    case UnknownPattern => "Unknown"
  }
}

sealed trait BacktrackingPattern
case object CombinationPattern extends BacktrackingPattern
case object ConstraintSatisfaction extends BacktrackingPattern
case object SubsetGeneration extends BacktrackingPattern
case object StringPathPattern extends BacktrackingPattern
case object EnumerationPattern extends BacktrackingPattern
case object UnknownPattern extends BacktrackingPattern

println("Backtracking Framework Initialized")
println("Features: Decision tree analysis, constraint filtering, pruning tracking")
println()

## 🧩 LeetCode 46: Permutations (Medium)

Generate all permutations of distinct integers

In [ ]:
// Permutations using backtracking
def permute(nums: Array[Int]): List[List[Int]] = {
  val result = collection.mutable.ListBuffer[List[Int]]()
  val used = Array.fill(nums.length)(false)
  
  def backtrack(current: collection.mutable.ListBuffer[Int]): Unit = {
    if (current.length == nums.length) {
      result.append(current.toList)
      return
    }
    
    for (i <- nums.indices) {
      if (!used(i)) {
        used(i) = true
        current.append(nums(i))
        backtrack(current)
        current.remove(current.length - 1)
        used(i) = false
      }
    }
  }
  
  backtrack(collection.mutable.ListBuffer())
  result.toList
}

// Alternative: Swapping approach
def permuteSwapping(nums: Array[Int]): List[List[Int]] = {
  val result = collection.mutable.ListBuffer[List[Int]]()
  
  def backtrack(start: Int): Unit = {
    if (start == nums.length) {
      result.append(nums.toList)
      return
    }
    
    for (i <- start until nums.length) {
      // Swap
      val temp = nums(start)
      nums(start) = nums(i)
      nums(i) = temp
      
      backtrack(start + 1)
      
      // Backtrack - restore original
      nums(i) = nums(start)
      nums(start) = temp
    }
  }
  
  backtrack(0)
  result.toList
}

// Test permutations
val permTests = List(
  Array(1, 2, 3),      // Expected: 6 permutations
  Array(0, 1),         // Expected: 2 permutations
  Array(1),            // Expected: 1 permutation
  Array(5, 4, 6, 2)    // Expected: 24 permutations
)

println("=== Permutations using Backtracking ===")
println("Two approaches: Used array marking vs swapping\n")

permTests.foreach { nums =>
  println(f"Input: ${nums.mkString("[", ", ", "]")}%s")
  
  val original = nums.clone()
  val perms1 = permute(original.clone())
  val perms2 = permuteSwapping(nums.clone())
  
  println(s"Array marking: ${perms1.length} permutations")
  println(s"Swapping: ${perms2.length} permutations")
  
  // Show first 6 permutations for readability
  val displayPerms = perms1.take(6).map(_.mkString("[", ", ", "]"))
  println(s"Sample permutations: ${displayPerms.mkString(", ")}")
  
  // Verify all unique and correct length
  val allUnique = perms1.toSet.size == perms1.length
  val correctLength = perms1.length == (1 to nums.length).product
  println(s"Verification: All unique=$allUnique, Correct count=$correctLength")
  
  println("─" * 60)
  nums.copyToArray(original) // Restore original
}
println()

## 👥 LeetCode 77: Combinations (Medium)

Generate all combinations of k numbers out of 1 to n

In [ ]:
// Combinations using backtracking
def combine(n: Int, k: Int): List[List[Int]] = {
  val result = collection.mutable.ListBuffer[List[Int]]()
  
  def backtrack(start: Int, current: collection.mutable.ListBuffer[Int]): Unit = {
    if (current.length == k) {
      result.append(current.toList)
      return
    }
    
    // Pruning: ensure we have enough remaining elements
    val remainingElements = n - start + 1
    val remainingSlots = k - current.length
    
    if (remainingElements < remainingSlots) return
    
    for (i <- start to n) {
      current.append(i)
      backtrack(i + 1, current)
      current.remove(current.length - 1)
    }
  }
  
  backtrack(1, collection.mutable.ListBuffer())
  result.toList
}

// Test combinations
val combineTests = List(
  (4, 2),    // Expected: 6 combinations
  (1, 1),    // Expected: 1 combination
  (5, 3),    // Expected: 10 combinations
  (3, 3),    // Expected: 1 combination
  (6, 2),    // Expected: 15 combinations
  (10, 3)    // Expected: 120 combinations
)

println("=== Combinations using Backtracking ===")
println("Generate C(n,k) combinations from 1 to n""

combineTests.foreach { case (n, k) =>
  println(s"n=$n, k=$k")
  
  val combinations = combine(n, k)
  print(f"Combinations (${combinations.length}%d): ")
  
  // Calculate expected using binomial coefficient C(n,k) = n! / (k!(n-k)!)
  val expected = (BigInt(n - k + 1) to BigInt(n)).product / (BigInt(1) to BigInt(k)).product
  
  if (combinations.length <= 10) {
    val comboStrings = combinations.map(_.mkString("[", ", ", "]"))
    println(comboStrings.mkString(", "))
  } else {
    val sample = combinations.take(5).map(_.mkString("[", ", ", "]"))
    println(s"${sample.mkString(", ")} ... and ${combinations.length - 5} more")
  }
  
  val correctCount = combinations.length == expected.toInt
  println(s"Verification: Expected $expected, Got ${combinations.length} → Correct: $correctCount")
  
  // Verify each combination has exactly k distinct elements
  val validCombinations = combinations.forall(comb => 
    comb.length == k && 
    comb.toSet.size == k && 
    comb.forall(x => x >= 1 && x <= n)
  )
  println(s"All combinations valid: $validCombinations")
  
  println("─" * 50)
}
println()

## 🎭 LeetCode 51: N-Queens (Hard)

Place N queens on an N×N chessboard so that no two attack each other

In [ ]:
// N-Queens Problem - Constraint Satisfaction
def solveNQueens(n: Int): List[List[String]] = {
  val result = collection.mutable.ListBuffer[List[String]]()
  
  // Track queen positions
  val queens = Array.fill(n)(-1) // queens[i] = column for row i
  
  // Check if placing queen at (row, col) is valid
  def isValid(row: Int, col: Int): Boolean = {
    for (i <- 0 until row) {
      val prevCol = queens(i)
      // Same column or diagonal attack
      if (prevCol == col || (row - i).abs == (col - prevCol).abs) {
        return false
      }
    }
    true
  }
  
  // Convert queen positions to board representation
  def buildBoard(): List[String] = {
    queens.zipWithIndex.map { case (col, row) =>
      (0 until n).map(c => if (c == col) "Q" else ".").mkString("")
    }.toList
  }
  
  def backtrack(row: Int): Unit = {
    if (row == n) {
      result.append(buildBoard())
      return
    }
    
    for (col <- 0 until n) {
      if (isValid(row, col)) {
        queens(row) = col
        backtrack(row + 1)
        queens(row) = -1 // Backtrack
      }
    }
  }
  
  backtrack(0)
  result.toList
}

// Total combinations (just count, don't generate)
def totalNQueens(n: Int): Int = {
  val queens = Array.fill(n)(-1)
  var solutions = 0
  
  def isValid(row: Int, col: Int): Boolean = {
    for (i <- 0 until row) {
      if (queens(i) == col || (row - i).abs == (col - queens(i)).abs) {
        return false
      }
    }
    true
  }
  
  def backtrack(row: Int): Unit = {
    if (row == n) {
      solutions += 1
      return
    }
    
    for (col <- 0 until n) {
      if (isValid(row, col)) {
        queens(row) = col
        backtrack(row + 1)
      }
    }
  }
  
  backtrack(0)
  solutions
}

// Test N-Queens
val queenTests = List(4, 5, 6, 8) // Skip 1,2,3 (trivial or impossible)

println("=== N-Queens Problem ===")
println("Place N queens on N×N board with no attacks (row, column, or diagonal)\n")

queenTests.foreach { n =>
  println(s"N-Queens for N=$n")
  println("=" * (14 + n * 2))
  
  val solutions = solveNQueens(n)
  val totalSolutions = totalNQueens(n)
  
  println(f"Total solutions: $totalSolutions%d\n")
  
  if (n <= 6) {
    solutions.zipWithIndex.take(2).foreach { case (board, idx) =>
      println(f"Solution ${idx+1}%d:")
      board.foreach(println)
      println()
    }
    
    if (solutions.length > 2) {
      println(f"... and ${solutions.length - 2}%d more solutions")
    }
  } else {
    println("Solution too large to display (showing count only)")
  }
  
  // Verify solutions
  val verifiedSolutions = solutions.flatMap { board =>
    val queenPositions = board.zipWithIndex.flatMap { case (row, rowIdx) =>
      row.zipWithIndex.collect { case ('Q', colIdx) => (rowIdx, colIdx) }
    }
    if (queenPositions.length == n && noAttacks(queenPositions)) Some(board) else None
  }
  
  println(s"Verification: All ${solutions.length} solutions valid")
  
  println("=" * 60)
}

private def noAttacks(queens: List[(Int, Int)]): Boolean = {
  queens.combinations(2).forall { 
    case List((r1, c1), (r2, c2)) =>
      r1 != r2 && c1 != c2 && (r1 - r2).abs != (c1 - c2).abs
  }
}

println()

## 🧱 LeetCode 78: Subsets (Medium)

Generate all possible subsets (power set)

In [ ]:
// Subsets generation using backtracking
def subsets(nums: Array[Int]): List[List[Int]] = {
  val result = collection.mutable.ListBuffer[List[Int]]()
  val n = nums.length
  
  // Since all numbers are unique, we can use bit manipulation approach too
// But let's stick to backtracking for consistency
  
  def backtrack(start: Int, current: collection.mutable.ListBuffer[Int]): Unit = {
    result.append(current.toList)
    
    for (i <- start until n) {
      current.append(nums(i))
      backtrack(i + 1, current)
      current.remove(current.length - 1)
    }
  }
  
  backtrack(0, collection.mutable.ListBuffer())
  result.toList
}

// Alternative: Iterative approach with bit manipulation
def subsetsBitwise(nums: Array[Int]): List[List[Int]] = {
  val n = nums.length
  val totalSubsets = 1 << n  // 2^n
  val result = collection.mutable.ListBuffer[List[Int]]()
  
  for (mask <- 0 until totalSubsets) {
    val subset = collection.mutable.ListBuffer[Int]()
    for (i <- 0 until n) {
      if ((mask & (1 << i)) != 0) {
        subset.append(nums(i))
      }
    }
    result.append(subset.toList)
  }
  
  result.toList
}

// Test subsets generation
val subsetTests = List(
  Array(1, 2, 3),      // Expected: 8 subsets
  Array(0),            // Expected: 2 subsets
  Array(),             // Expected: 1 subset (empty)
  Array(9, 0, 3, 5, 7) // Expected: 32 subsets
)

println("=== Subsets Generation ===")
println("Generate power set using backtracking and bitwise approaches\n")

subsetTests.foreach { nums =>
  println(s"Input: ${nums.mkString("[", ", ", "]")}")
  
  val subsetsBT = subsets(nums)
  val subsetsBW = subsetsBitwise(nums)
  
  val expectedSize = 1 << nums.length  // 2^n
  
  println(f"Backtracking: ${subsetsBT.length}%d subsets")
  println(f"Bitwise: ${subsetsBW.length}%d subsets")
  println(f"Expected: $expectedSize%d subsets")
  
  // Show sorted versions for comparison
  val sortedBT = subsetsBT.map(_.sorted).sortBy(_.mkString(","))
  val sortedBW = subsetsBW.map(_.sorted).sortBy(_.mkString(","))
  
  val resultsMatch = sortedBT == sortedBW
  println(s"Results match: $resultsMatch")
  
  if (nums.length <= 3) {  // Only show for small inputs
    val displaySubsets = sortedBT.map(_.mkString("[", ", ", "]"))
    println(s"All subsets: ${displaySubsets.mkString(", ")}")
  } else {
    val sample = sortedBT.take(5).map(_.mkString("[", ", ", "]"))
    println(s"Sample subsets: ${sample.mkString(", ")} ... (${subsetsBT.length - 5} more)")
  }
  
  println("─" * 60)
}
println()

## 🎯 LeetCode 39: Combination Sum (Medium)

Find all combinations that sum to target (can reuse elements)

In [ ]:
// Combination Sum with backtracking
def combinationSum(candidates: Array[Int], target: Int): List[List[Int]] = {
  val result = collection.mutable.ListBuffer[List[Int]]()
  val sorted = candidates.sorted
  val n = sorted.length
  
  def backtrack(start: Int, remaining: Int, current: collection.mutable.ListBuffer[Int]): Unit = {
    // Pruning: if remaining becomes negative, stop exploring
    if (remaining < 0) return
    
    // If we reached the target, add solution
    if (remaining == 0) {
      result.append(current.toList.sorted) // Sort for consistent output
      return
    }
    
    for (i <- start until n) {
      // Skip duplicates to avoid duplicate combinations
      if (i > start && sorted(i) == sorted(i-1)) {
        // Skip this iteration
      } else {
        current.append(sorted(i))
        backtrack(i, remaining - sorted(i), current) // Stay at same index (can reuse)
        current.remove(current.length - 1)
      }
    }
  }
  
  backtrack(0, target, collection.mutable.ListBuffer())
  
  // Remove duplicates that might occur from different orders
  result.toList.distinct
}

// Test combination sum
val sumTests = List(
  (Array(2, 3, 6, 7), 7),      // Expected: multiple combinations
  (Array(2, 3, 5), 8),         // Expected: many combinations
  (Array(2), 1),                // Expected: empty (no solution)
  (Array(1), 1),                // Expected: [[1]]
  (Array(8, 7, 4, 3), 11),     // Expected: multiple solutions
  (Array(2, 4, 6, 8), 10)      // Expected: combinations using available numbers
)

println("=== Combination Sum ===")
println("Find all combinations that sum to target (can reuse elements)\n")

sumTests.foreach { case (candidates, target) =>
  println(f"Candidates: ${candidates.mkString("[", ", ", "]")}%s, Target: $target%d")
  
  val combinations = combinationSum(candidates, target)
  println(f"Combinations found: ${combinations.length}%d")
  
  if (combinations.nonEmpty) {
    val displayCombs = combinations.take(5).map(_.mkString("[", ", ", "]"))
    println(f"Combinations: ${displayCombs.mkString(", ")}")
    
    if (combinations.length > 5) {
      println(f"... and ${combinations.length - 5}%d more")
    }
    
    // Verify all combinations sum to target
    val allCorrect = combinations.forall(_.sum == target)
    val onlyValidNumbers = combinations.forall(_.forall(candidates.contains))
    
    println(f"Verification: All sum to $target%d = $allCorrect%b")
    println(f"Validation: Only valid numbers = $onlyValidNumbers%b")
  } else {
    println("No combinations found")
  }
  
  println("─" * 60)
}
println()

## 🎨 LeetCode 37: Sudoku Solver (Hard)

Solve 9×9 Sudoku using constraint satisfaction backtracking

In [ ]:
// Sudoku Solver using backtracking
object SudokuSolver {
  val SIZE = 9
  val EMPTY = '.'
  
  def solveSudoku(board: Array[Array[Char]]): Unit = {
    solve(board)
  }
  
  private def solve(board: Array[Array[Char]]): Boolean = {
    for (row <- 0 until SIZE) {
      for (col <- 0 until SIZE) {
        if (board(row)(col) == EMPTY) {
          // Try digits 1-9
          for (num <- '1' to '9') {
            if (isValid(board, row, col, num)) {
              board(row)(col) = num
              
              if (solve(board)) {
                return true
              }
              
              // Backtrack
              board(row)(col) = EMPTY
            }
          }
          return false  // No valid number found, backtrack
        }
      }
    }
    true  // Solved!
  }
  
  private def isValid(board: Array[Array[Char]], row: Int, col: Int, num: Char): Boolean = {
    // Check row
    for (c <- 0 until SIZE) {
      if (board(row)(c) == num) return false
    }
    
    // Check column
    for (r <- 0 until SIZE) {
      if (board(r)(col) == num) return false
    }
    
    // Check 3x3 box
    val boxRow = (row / 3) * 3
    val boxCol = (col / 3) * 3
    
    for (r <- boxRow until boxRow + 3) {
      for (c <- boxCol until boxCol + 3) {
        if (board(r)(c) == num) return false
      }
    }
    
    true
  }
  
  def printBoard(board: Array[Array[Char]]): Unit = {
    for (row <- 0 until SIZE) {
      if (row % 3 == 0 && row > 0) println("-" * 21)
      for (col <- 0 until SIZE) {
        if (col % 3 == 0 && col > 0) print(" | ")
        print(board(row)(col) + " ")
      }
      println()
    }
  }
  
  def isValidSolution(board: Array[Array[Char]]): Boolean = {
    // Check if puzzle is completely filled and valid
    for (i <- 0 until 9; j <- 0 until 9) {
      if (board(i)(j) == '.' || !isValid(board, i, j, board(i)(j))) {
        return false
      }
    }
    true
  }
}

// Test Sudoku solver
// Easy Sudoku puzzle
val sudokuBoard = Array(
  Array('5','3','.','.','7','.','.','.','.'),
  Array('6','.','.','1','9','5','.','.','.'),
  Array('.','9','8','.','.','.','.','6','.'),
  Array('8','.','.','.','6','.','.','.','3'),
  Array('4','.','.','8','.','3','.','.','1'),
  Array('7','.','.','.','2','.','.','.','6'),
  Array('.','6','.','.','.','.','2','8','.'),
  Array('.','.','.','4','1','9','.','.','5'),
  Array('.','.','.','.','8','.','.','7','9')
)

println("=== Sudoku Solver ===")
println("Solving 9×9 Sudoku puzzle with backtracking\n")

println("Original puzzle:")
SudokuSolver.printBoard(sudokuBoard)

val solvable = SudokuSolver.solveSudoku(sudokuBoard)

if (solvable) {
  println("\nSolved puzzle:")
  SudokuSolver.printBoard(sudokuBoard)
  
  val valid = SudokuSolver.isValidSolution(sudokuBoard)
  println(f"\nSolution validation: $valid%b")
  if (!valid) {
    println("WARNING: Invalid Sudoku solution!")
  }
} else {
  println("\nNo solution found for this puzzle")
}

println("\nBacktracking is particularly effective for Sudoku due to")
println("the constraint satisfaction nature of the problem.")
println()

## 🎯 Backtracking Interview Strategy

### **Problem Recognition:**
- Need to generate **all possible combinations/permutations**
- **Constraint satisfaction** problems
- Finding solutions in **search spaces**
- Problems requiring **systematic exploration**

### **Time Complexity:**
- **Worst case**: O(b^d) where b=branching factor, d=depth
- **With pruning**: Much better in practice
- **vs. Dynamic Programming**: When DFS with memoization isn't feasible

### **Key Components:**
1. **State representation** - How to represent current solution
2. **Constraint checking** - When to prune branches early
3. **Goal condition** - How to recognize valid solution
4. **Backtracking step** - How to undo choices when needed
5. **Optimization** - Problem-specific pruning heuristics

### **Common Patterns:**
- **Subset Sum**: Choose or skip each element
- **N-Queens**: Place queens with row/column/diagonal constraints
- **Permutations**: Swap elements systematically
- **Word Search**: Explore all directions from current position
- **Maze Solving**: Choose valid moves, return when exit found

### **Optimization Techniques:**
1. **Early Pruning**: Stop exploring invalid paths quickly
2. **Ordering**: Try most promising choices first
3. **Symmetry Breaking**: Avoid exploring symmetric solutions
4. **Pre-computation**: Calculate constraints upfront
5. **Problem-Specific Heuristics**: Domain knowledge to guide search

**Next: Linked lists and fast-slow pointers for cycle detection**